# DCGAN

Based in the proposed paper : https://arxiv.org/abs/1511.06434

## Install Libraries

In [3]:
!pip install pytorch-msssim
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 2.1 MB/s eta 0:00:0000:0100:01


## Import Libraries

In [5]:
import torch
import torch.nn.functional as F
from torchvision import models
from pytorch_msssim import ssim  # For SSIM metric
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
import torch.optim as optim
import logging

In [6]:
# Define device as cuda if available

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
 # Define transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

In [9]:
# Dataset

train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
dataloader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)

100%|██████████████████████████████████████| 26.4M/26.4M [00:15<00:00, 1.74MB/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|███████████████████████████████████████| 29.5k/29.5k [00:00<00:00, 930kB/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████████████████████████████████| 4.42M/4.42M [00:02<00:00, 1.70MB/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████████████████████████████████| 5.15k/5.15k [00:00<00:00, 6.70MB/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [ ]:
# number of channels
nc=1

# input noise dimension
nz = 100

# number of generator filters
ngf = 64

#number of discriminator filters
ndf = 64

In [ ]:
# Baseline from LAB 5(INM705 Deep Learning for Image Analysis)

class Discriminator(nn.Module):
    def __init__(self, nc, ndf):
        super(Discriminator, self).__init__()
        self.nc = nc
        self.ndf = ndf
        self.model = nn.Sequential(
            nn.Conv2d(self.nc, self.ndf, 5, stride=2, padding=2),
            nn.LeakyReLU(0.3),
            nn.Dropout(0.3),
            nn.Conv2d(self.ndf,self.ndf*2, 5, stride=2, padding=2),
            nn.LeakyReLU(0.3),
            nn.Dropout(0.3),
            nn.Flatten(),
            nn.Linear((self.ndf*2)*7*7, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# Baseline from LAB 5 (INM705 Deep Learning for Image Analysis)

class Generator(nn.Module):
      def __init__(self, nc, ngf, nz):
        super(Generator, self).__init__()
        self.nc = nc
        self.ngf = ngf
        self.nz = nz
        self.model = nn.Sequential(
            nn.Linear(100, 7*7*(self.ngf*4), bias=False),
            nn.BatchNorm1d(7*7*(self.ngf*4)),
            nn.LeakyReLU(0.3),
            nn.Unflatten(self.nc, (self.ngf*4, 7, 7)),
            nn.ConvTranspose2d(self.ngf*4, self.ngf*2, 5, stride=1, padding=2, bias=False),
            nn.BatchNorm2d(self.ngf*2),
            nn.LeakyReLU(0.3),
            nn.ConvTranspose2d(self.ngf*2, self.ngf, 5, stride=2, padding=2, output_padding=1, bias=False),
            nn.BatchNorm2d(self.ngf),
            nn.LeakyReLU(0.3),
            nn.ConvTranspose2d(self.ngf, self.nc, 5, stride=2, padding=2, output_padding=1, bias=False),
            nn.Tanh()
        )

      def forward(self, x):
        return self.model(x)


In [ ]:
# Initialize and assign models to the device

D = Discriminator(nc,ndf).to(device)
G = Generator(nc,ngf,nz).to(device)

print(D)
print(G)

In [ ]:
# Initialize optimizers

optimizerD = optim.Adam(D.parameters(), lr=1e-4)
optimizerG = optim.Adam(G.parameters(), lr=1e-4)

# Define loss
criterion = nn.BCELoss()

## Function to save images generated

In [ ]:
def save_images(epoch):
    os.makedirs('outputs', exist_ok=True)
    fixed_noise = torch.randn(32, 100, device=device)
    fake_images = G(fixed_noise).detach().cpu()
    save_image(fake_images, f'outputs/epoch_{epoch}.png', nrow=8, normalize=True)

## Metrics definition

The following metrics are based on the following source: https://medium.com@heyamit10/pytorch-implementation-of-common-gan-metrics-86f993f6e737

Changes were made to the implementation of the methods in order to modify them for the dataset used in this work

In [ ]:
# KID (Kernel Inception Distance)

def polynomial_mmd(x, y, degree=3, gamma=None, coef0=1):
    x = x.reshape(x.size(0), -1)  # Reshape to 2D
    y = y.reshape(y.size(0), -1)  # Reshape to 2D
    if gamma is None:
        gamma = 1.0 / x.shape[1]
    kernel_xx = (gamma * x.mm(x.t()) + coef0) ** degree
    kernel_yy = (gamma * y.mm(y.t()) + coef0) ** degree
    kernel_xy = (gamma * x.mm(y.t()) + coef0) ** degree
    return kernel_xx.mean() + kernel_yy.mean() - 2 * kernel_xy.mean()



def kernel_inception_distance(real_features, generated_features):
    real_features, generated_features = torch.tensor(real_features), torch.tensor(generated_features)
    return polynomial_mmd(real_features, generated_features

In [ ]:
# Load pre-trained Inception model

inception = models.inception_v3(pretrained=True).eval()
inception.fc = torch.nn.Identity()  # Remove last layer

In [ ]:
# Precision and recall

def get_features(images, model, batch_size=32):
    features = []
    for i in range(0, len(images), batch_size):
        batch = images[i:i + batch_size].cuda()
        with torch.no_grad():
            batch_features = model(batch).cpu().numpy()
        features.append(batch_features)
    return np.concatenate(features)

def compute_precision_recall(real_features, generated_features, k=5):
    real_features = real_features.reshape(real_features.size(0), -1).cpu().numpy()  # Reshape to 2D and convert to numpy
    generated_features = generated_features.reshape(generated_features.size(0), -1).cpu().numpy()  # Reshape to 2D and convert to numpy
    # Initialize FAISS index for nearest neighbors
    index = faiss.IndexFlatL2(real_features.shape[1])
    index.add(real_features)

    # Precision: Nearest neighbors for generated samples in real set
    D, I = index.search(generated_features, k)
    precision = np.mean([np.any(i in I for i in range(len(real_features))) for _ in D])

    # Recall: Nearest neighbors for real samples in generated set
    index.reset()
    index.add(generated_features)
    D, I = index.search(real_features, k)
    recall = np.mean([np.any(i in I for i in range(len(generated_features))) for _ in D])

    return precision, recall